In [207]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from datetime import timedelta
from datetime import datetime

%run web_scraping.ipynb
%run useful_functions.ipynb

In [ ]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

In [199]:
#shows_to_run = ["https://www.bbc.co.uk/programmes/m00008zy/episodes/guide", 
#                "https://www.bbc.co.uk/programmes/b03hjfww/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b006wkh9/episodes/guide",
#                "https://www.bbc.co.uk/programmes/p04m6srg/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m000m5bw/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m0016zh5/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072lb2/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072q60/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00p4l4t/episodes/guide"]
#                "https://www.bbc.co.uk/programmes/b0072l4x/episodes/guide",

temp_show_broadcast_details = pd.DataFrame()

for show_url in shows['Show URL']:
    show_instance = Show(show_url)
    if shows.empty or show_instance.show_key not in shows['Show Key'].unique():
        shows = pd.concat([shows,show_instance.show_details])
    temp_show_broadcast_details = pd.concat([temp_show_broadcast_details,show_instance.broadcasts])
    
temp_show_broadcast_details['Datetime Key'] = datetime.now()
temp_show_broadcast_details['Is New Data'] = True

show_broadcast_details = pd.concat([show_broadcast_details,temp_show_broadcast_details])
show_broadcast_details = show_broadcast_details.groupby(allbut(show_broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
show_broadcast_details = show_broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

In [170]:
new_broadcast_data = show_broadcast_details[(show_broadcast_details["Is Broadcast Upcoming"]==False) & (show_broadcast_details["Is New Data"]==True)]#[0:10]
temp_broadcast_details = pd.DataFrame()
temp_broadcast_playlists = pd.DataFrame()

for i, [broadcast_url,broadcast_key] in enumerate(new_broadcast_data[['Broadcast URL','Broadcast Key']].values):
    broadcast_instance = Playlist(broadcast_url)
    if i % 10 == 0:
        print(f'Broadcast Number {1+i} of {len(new_broadcast_data.index)}')
    temp_broadcast_details = pd.concat([temp_broadcast_details,broadcast_instance.broadcast_details])
    temp_broadcast_playlists = pd.concat([temp_broadcast_playlists,broadcast_instance.playlist])
    
temp_broadcast_details['Datetime Key'] = datetime.now()
temp_broadcast_details['Is New Data'] = True
temp_broadcast_playlists['Datetime Key'] = datetime.now()
temp_broadcast_playlists['Is New Data'] = True

broadcast_details = pd.concat([broadcast_details,temp_broadcast_details])
broadcast_details = broadcast_details.groupby(allbut(broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details = broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

broadcast_playlists = pd.concat([broadcast_playlists,temp_broadcast_playlists])
broadcast_playlists = broadcast_playlists.groupby(allbut(broadcast_playlists.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists = broadcast_playlists.groupby(["Broadcast Key", "Broadcast Position"]).apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)


Broadcast Number 1 of 296
Broadcast Number 2 of 296
Broadcast Number 3 of 296
Broadcast Number 4 of 296
Broadcast Number 5 of 296
Broadcast Number 6 of 296
Broadcast Number 7 of 296
Broadcast Number 8 of 296
Broadcast Number 9 of 296
Broadcast Number 10 of 296
Broadcast Number 11 of 296
Broadcast Number 12 of 296
Broadcast Number 13 of 296
Broadcast Number 14 of 296
Broadcast Number 15 of 296
Broadcast Number 16 of 296
Broadcast Number 17 of 296
Broadcast Number 18 of 296
Broadcast Number 19 of 296
Broadcast Number 20 of 296
Broadcast Number 21 of 296
Broadcast Number 22 of 296
Broadcast Number 23 of 296
Broadcast Number 24 of 296
Broadcast Number 25 of 296
Broadcast Number 26 of 296
Broadcast Number 27 of 296
Broadcast Number 28 of 296
Broadcast Number 29 of 296
Broadcast Number 30 of 296
Broadcast Number 31 of 296
Broadcast Number 32 of 296
Broadcast Number 33 of 296
Broadcast Number 34 of 296
Broadcast Number 35 of 296
Broadcast Number 36 of 296
Broadcast Number 37 of 296
Broadcast 

In [187]:
#show_broadcast_details['Is New Data'] = False
#broadcast_details['Is New Data'] = False
#broadcast_playlists['Is New Data'] = False

shows.to_pickle("../data/shows.pkl")
show_broadcast_details.to_pickle("../data/show_broadcast_details.pkl")
broadcast_details.to_pickle("../data/broadcast_details.pkl")
broadcast_playlists.to_pickle("../data/broadcast_playlists.pkl")

In [206]:
mask_2022 = broadcast_details['Service Date'].apply(lambda date: date.year) == 2022

(
broadcast_details[mask_2022]
.merge(broadcast_playlists, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

,Show Title,Broadcast Banner,Broadcast Long Description,Service Image URL,Service Day,Service Date,Service Time,Broadcast Key,Datetime Key Broadcast Details,Is New Data Broadcast Details,...,Broadcast Image URL,Broadcast URL,Broadcast Title,Broadcast Short Description,Is Broadcast Upcoming,Show Key,Datetime Key Show Details,Is New Data Show Details,Show URL,Show Name
907,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,4592383511527557379,2022-12-10 17:03:33.752876,True,...,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,-2655158488449224554,2022-12-10 17:01:21.601036,True,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
908,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,4592383511527557379,2022-12-10 17:03:33.752876,True,...,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,-2655158488449224554,2022-12-10 17:01:21.601036,True,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
909,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,4592383511527557379,2022-12-10 17:03:33.752876,True,...,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,-2655158488449224554,2022-12-10 17:01:21.601036,True,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
910,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,4592383511527557379,2022-12-10 17:03:33.752876,True,...,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,-2655158488449224554,2022-12-10 17:01:21.601036,True,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
911,Radio 1's Indie Show with Jack Saunders,The best in indie and alternative,Two indie fans go head to head in the ultimate...,https://programmes.files.bbci.co.uk/programmes...,Sun 04 Dec 2022,2022-12-04,21:00,4592383511527557379,2022-12-10 17:03:33.752876,True,...,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m001fl2g,The best in indie and alternative,Two indie fans go head to head in a game of Ho...,False,-2655158488449224554,2022-12-10 17:01:21.601036,True,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,Radio 1's Indie Show with Jack Saunders,Twin Atlantic,Twin Atlantic talk new album 'Transparency' an...,https://programmes.files.bbci.co.uk/programmes...,Thu 13 Jan 2022,2022-01-13,20:00,-351739348369292085,2022-12-10 17:03:33.752876,True,...,https://ichef.bbci.co.uk/images/ic/320x180/p0c...,https://www.bbc.co.uk/programmes/m00136gz,Twin Atlantic,Twin Atlantic talk new album 'Transparency' an...,False,-2655158488449224554,2022-12-10 17:01:21.601036,True,https://www.bbc.co.uk/programmes/m00008zy/epis...,Radio 1's Indie Show with Jack Saunders
485,Radio 1's Indie Show with 

In [196]:
shows
show_broadcast_details
broadcast_details
#broadcast_playlists

,Show Title,Broadcast Banner,Broadcast Long Description,Service Image URL,Service Day,Service Date,Service Time,Broadcast Key,Datetime Key,Is New Data
0,Radio 1's Indie Show with Jack Saunders,Next Wave 2019,Jack picks his favourite discoveries from Next...,https://programmes.files.bbci.co.uk/programmes...,Mon 23 Dec 2019,2019-12-23,23:00,-9135362881644799146,2022-12-10 17:03:33.752876,True
1,Radio 1's Indie Show with Jack Saunders,Next Wave: Home Counties,Jack introduces you to Bristol 5-piece Home Co...,https://programmes.files.bbci.co.uk/programmes...,Thu 21 Jan 2021,2021-01-21,22:00,-8938032348107244964,2022-12-10 17:03:33.752876,True
2,Radio 1's Indie Show with Jack Saunders,Demob Happy Live Session,Brighton-based trio Demob Happy perform new mu...,https://programmes.files.bbci.co.uk/programmes...,Tue 7 Jan 2020,2020-01-07,23:00,-8871385001390754473,2022-12-10 17:03:33.752876,True
3,Radio 1's Indie Show with Jack Saunders,Nels Hylton sits in for Jack,Nels Hylton with all things indie and alternat...,https://programmes.files.bbci.co.uk/programmes...,Thu 28 Jul 2022,2022-07-28,20:00,-8807100618129792533,2022-12-10 17:03:33.752876,True
4,Radio 1's Indie Show with Jack Saunders,The A-Z of Indie: Justice,Jack continues to build the ultimate guide to ...,https://programmes.files.bbci.co.uk/programmes...,Wed 29 Apr 2020,2020-04-29,23:00,-8716048988866920353,2022-12-10 17:03:33.752876,True
...,...,...,...,...,...,...,...,...,...,...
291,Radio 1's Indie Show with Jack Saunders,The A-Z of Indie: Frank Zappa,Jack finishes the ultimate guide to indie/alte...,https://programmes.files.bbci.co.uk/programmes...,Wed 24 Jun 2020,2020-06-24,23:00,9076497870158109636,2022-12-10 17:03:33.752876,True
292,Radio 1's Indie Show with Jack Saunders,Sundara Karma,Oscar from Sundara Karma joins Jack for the In...,https://programmes.files.bbci.co.uk/programmes...,Thu 24 Mar 2022,2022-03-24,20:00,9098165914593094156,2022-12-10 17:03:33.752876,True
293,Radio 1's Indie Show with Jack Saunders,Blossoms,Tom and Joe from Blossoms join Jack for the In...,https://programmes.files.bbci.co.uk/programmes...,Thu 21 Apr 2022,2022-04-21,20:00,9168535692854716338,2022-12-10 17:03:33.752876,True
294,Radio 1's Indie Show with Jack Saunders,Mince Pies with Jack Saunders!,Jack invites some special guests including Fon...,https://programmes.files.bbci.co.uk/programmes...,Tue 24 Dec 2019,2019-12-24,23:00,9169135045801013394,2022-12-10 17:03:33.752876,True


,Artist Image URL,Artist Name,Track Name,Record Label,Track Number,Broadcast Position,Broadcast Key,Datetime Key,Is New Data
0,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Declan Welsh & The Decadent West,No Fun,Modern Sky,,1.0,-9135362881644799146,2022-12-10 17:03:33.754326,True
1,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Spyres,Otherside,Spyres,,2.0,-9135362881644799146,2022-12-10 17:03:33.754326,True
2,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,The Howlers,La Dolce Vita,,,3.0,-9135362881644799146,2022-12-10 17:03:33.754326,True
3,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Badgirl$,Stella,,,4.0,-9135362881644799146,2022-12-10 17:03:33.754326,True
4,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,YUNGBLUD,"Loner (Radio 1 Next Wave Gig, 21 March 2019)",,,5.0,-9135362881644799146,2022-12-10 17:03:33.754326,True
...,...,...,...,...,...,...,...,...,...
9236,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Wargasm,Spit,Angry Songs For Sad People,,28.0,9205832051522900160,2022-12-10 17:03:33.754326,True
9237,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Travis Barker,Forever,DTA/Elektra,,29.0,9205832051522900160,2022-12-10 17:03:33.754326,True
9238,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,FEVER 333,Supremacy,,,30.0,9205832051522900160,2022-12-10 17:03:33.754326,True
9239,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Nova Twins,Taxi,Nova Twins Ltd / 333 Wreckords,,31.0,9205832051522900160,2022-12-10 17:03:33.754326,True
